Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?

I'm going to attempt to predict region based on Thanksgiving foods. I might also try to predict whether someone is a Black Friday shopper based on foods.
- [ ] Is your problem regression or classification?
Classification
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
South Atlantic        0.208711
Middle Atlantic       0.161525
East North Central    0.130672
Pacific               0.123412
West South Central    0.098004
West North Central    0.079855
East South Central    0.065336
New England           0.050817
Mountain              0.045372
No                    0.036298
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
    I'm using a random split based on sklearn train_test_split
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
- [ ] Begin to clean and explore your data.

Working on that! I have a lot of data cleaning to do.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?
I don't think any of my features will leak data. I will exclude the 'ID' feature since that just indicates the user who completed the survey, and I'm excluding any of the "other" features for the sake of simplicity. 